In [1]:
import datetime
from pysolar.solar import *


#This code is to process the raw NILU-UV data. It applies the drift and cosine correction factor and calculates SZA based on location and time.


#reading drift file
filename = '../Data/NILU115/drift.15'
infile = open(filename, 'r')
drift_data = infile.read()
infile.close()

In [2]:
#Hoboken latitude and longitude
latitude = 40.74287154007815
longitude = -74.02740216636893

In [3]:
#Entering the choosen channal numbers for calculating their ratio
numerator = 3    # input("Enter channel number for numerator: ")
denominator = 1  #input("Enter channel number for denominator: ")

In [4]:
#defining empty lists
drift_year = []
drift_month = []
drift_day = []
drift_daynumber = []
drift_ch1 = []
drift_ch2 = []
drift_ch3 = []
drift_ch4 = []
drift_ch5 = []
drift_ch6 = []
cosine_corr_ch1 = []
cosine_corr_ch2 = []
cosine_corr_ch3 = []
cosine_corr_ch4 = []
cosine_corr_ch5 = []
cosine_corr_ch6 = []
cosine_corr_SZA = []
drift_yyyymmdd = []


In [5]:
# splitting drift data into rows
row1 = drift_data.split('\n')

In [6]:
# splitting drift data into columns (lists)
for i in range(1,len(row1),1): 
    element = row1[i].split() 
    drift_year.append(element[0]) 
    drift_month.append(element[1]) 
    drift_day.append(element[2]) 
    drift_daynumber.append(element[3]) 
    drift_ch1.append(element[4]) 
    drift_ch2.append(element[5]) 
    drift_ch3.append(element[6]) 
    drift_ch4.append(element[7]) 
    drift_ch5.append(element[8]) 
    drift_ch6.append(element[9]) 
    drift_yyyymmdd.append(datetime.datetime(int(element[0]),int(element[1]), int(element[2]),0,0,0,0,tzinfo=datetime.timezone.utc)) 
    element = [] 
    
# dates of calibrations are saved in the list drift_yyyymmdd,calibration time (hh mm ss microsec) set to 0,
# timezone to utc in order to compare it to measurement time

In [7]:
#Reading Cosine_NILUV_4115.txt file, which contains the cosine correction for the instrument

cosine_correction_file = 'Cosine_NILUV_4115.txt'
infile = open(cosine_correction_file, 'r')
cosine_correction_data = infile.read()
infile.close()

In [8]:
# splitting drift data into rows
NILU_file_row = cosine_correction_data.split('\n')

# splitting drift data into columns (lists)
for i in range(12,len(NILU_file_row)-1,1): 
    element = NILU_file_row[i].split() 
    if i%2==0:
        cosine_corr_SZA.append(element[0])
        cosine_corr_ch1.append(element[1])
        cosine_corr_ch2.append(element[2])
        cosine_corr_ch3.append(element[4])
        cosine_corr_ch4.append(element[4])
        cosine_corr_ch5.append(element[5])
        cosine_corr_ch6.append(element[6])              

In [9]:
# Reading the input files which contains the measured data and getting rid of erronous lines/data.

import os

file = open('./NILU-OUT/temp/nilu-dirty.txt', 'w')

# Set the folder that contains the data files

for filename in os.listdir('../Data/NILU115/2019'):
    with open(os.path.join('../Data/NILU115/2019', filename), 'r', errors='ignore') as raw_input:
        text = raw_input.read()
        file.write(text)
        
file.close()
        
    
file = open('./NILU-OUT/temp/nilu-CLEAN.txt', 'w')

with open('./NILU-OUT/temp/nilu-dirty.txt') as temp:
    for line in temp:
        if len(line) == 80:
            file.write(line)

file.close()
    


In [10]:
#defining empty lists
date_list = []
time_list = []
doy_list = []

#M_ch# is drift corrected, without the cosine correction 
M_ch1 = []
M_ch2 = []
M_ch3 = []
M_ch4 = []
M_ch5 = []
M_ch6 = []
ratio = []
solar_angle = []

In [11]:
#splitting data into rows
with open('./NILU-OUT/temp/nilu-CLEAN.txt', 'r') as temp:
    data_measured = temp.read()

row2 = data_measured.split('\n')

In [12]:
for i in range(len(row2)-1):
    #splitting rows into elements 
    element = row2[i].split(' ')
    
    #Getting rid of dark (low irrad) data and 0 denominatiors.
    if float(element[2]) > 170 and float(element[6]) > 1 and float(element[4]) > 1:
    
    #creating compareble date and time format
        date = element[0]
        time = element[1]
        yyyy = date[0:4]
        mm = date[4:6]
        dd = date[6:8]
        hh = time[0:2]
        minmin = time[2:4]
    
    #creating date and time list for output file
        date_list.append(date)
        time_list.append(time)
        doy_list.append(datetime.datetime(int(yyyy), int(mm), int(dd)).timetuple().tm_yday)

    
    #convert measurement_time into comparable format, measurement_time is an input variable for the SZA calculator
    #check the time zone format of the input file and the timezone of "measurement_time"
    #both supposed to be in utc format
        measurement_time = datetime.datetime(int(yyyy),int(mm),int(dd),int(hh),int(minmin),0,0,tzinfo=datetime.timezone.utc)
        j = 1
              
    #checking the closest rel. calibration date
        while measurement_time > drift_yyyymmdd[j]:
            j = j + 1
        else:
        #dividing the measured channel data with the corresponding drift number, normalizing with a_i and b_i
            ch1 = float(element[2])/(float(drift_ch1[j]))
            ch2 = float(element[3])/(float(drift_ch2[j]))  
            ch3 = float(element[4])/(float(drift_ch3[j]))
            ch4 = float(element[5])/(float(drift_ch4[j]))
            ch5 = float(element[6])/(float(drift_ch5[j]))
        #ch6 = float(element[7])/float(drift_ch6[j]))
        
        #calculating the sz angle
        # altitude in this module meand elevation angle
            altitude = get_altitude(latitude, longitude, measurement_time)
            solar_angle_temp = 90 - altitude
            solar_angle.append(solar_angle_temp)
        
        #attaching the calculated drifted and normalized number into a list
            M_ch1.append(ch1)
            M_ch2.append(ch2)
            M_ch3.append(ch3)
            M_ch4.append(ch4)
            M_ch5.append(ch5)
        #drifted_ch6.append(ch6)
        
        #calculating the ratio
        #    M_temp = [ch1,ch2,ch3,ch4,ch5]
        #    ratio_temp = M_temp[int(numerator)-1]/M_temp[int(denominator)-1]
        #    ratio.append(ratio_temp)
 
        
        #days between the measurement and the next calibration
            time_delta1 = drift_yyyymmdd[j] - measurement_time    
        element = []
    
   
    

    
    

In [13]:
final_ch1 = []
final_ch2 = []
final_ch3 = []
final_ch4 = []
final_ch5 = []


#Correcting the channel data with the cosine correction
for i in range(len(M_ch1)):
    j = 0
    while (float(solar_angle[i]) - float(cosine_corr_SZA[j])) >= 0.5 and 90 > float(solar_angle[i]):
        j = j + 1
#        print(solar_angle[i],'\t',cosine_corr_SZA[j])
    else:
        final_ch1.append(float(M_ch1[i])/float(cosine_corr_ch1[j]))
        final_ch2.append(float(M_ch2[i])/float(cosine_corr_ch2[j]))
        final_ch3.append(float(M_ch3[i])/float(cosine_corr_ch3[j]))
        final_ch4.append(float(M_ch4[i])/float(cosine_corr_ch4[j]))
        final_ch5.append(float(M_ch5[i])/float(cosine_corr_ch5[j]))
        #dividing channel3 by channel1
        ratio.append((float(M_ch3[i])/float(cosine_corr_ch3[j]))/(float(M_ch1[i])/float(cosine_corr_ch1[j])))
    #print(solar_angle[i],'\t',final_ch5[i],'\t',M_ch5[i])

In [14]:
# Writing  output file
file2 = open('./NILU-OUT/processed_nilu_2019.txt', 'w')

for i in range(len(final_ch1)-1):
    if solar_angle[i] <= 70 and solar_angle[i] > 0:
        #print(date_list[i],'\t',time_list[i],'\t',f"{solar_angle[i]:.3}",'\t',f"{M_ch5[i]:.10}", '\t' ,f"{ratio[i]:.10}",'\t',file=file2)
        print(f"{solar_angle[i]:.5}", ' ',f"{final_ch5[i]:.10}", ' ' ,f"{ratio[i]:.10}", ' ' ,date_list[i], ' ' ,doy_list[i], ' ' ,time_list[i], file=file2)
file2.close()        
        
# Output txt file contains the following columns: sza, CH5, CH3/CH1, yyyymmdd, doy, hhmm

In [15]:
# file3 = open('./NILU-OUT/timestamp.txt', 'w')

# for i in range(len(M_ch1)-1):
#     if solar_angle[i] <= 70 and solar_angle[i] > 0:
#         print(date_list[i],' ',time_list[i], ' ', doy_list[i], f"{solar_angle[i]:.5}", file=file3)
# file3.close()
    
# Output txt file contains the following columns: yyyymmhh, hhmm, sza